In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from causalimpact import CausalImpact
import warnings
warnings.filterwarnings("ignore")

# --------------------------
# Helper function to parse summary text
def parse_summary(summary_text):
    """
    Parse the CausalImpact summary text and return key metrics.
    Returns a dictionary with:
      - abs_effect_avg: the average absolute effect estimate
      - abs_ci_lower: lower bound of the 95% CI for the absolute effect (average)
      - abs_ci_upper: upper bound of the 95% CI for the absolute effect (average)
    """
    abs_effect_avg, abs_ci_lower, abs_ci_upper = None, None, None
    current_section = 'main'
    lines = summary_text.split('\n')
    
    for line in lines:
        # Look for the absolute effect section
        if 'Absolute effect (s.d.)' in line:
            current_section = 'absolute'
            abs_effect_match = re.search(r'(-?\d+\.\d+)\s+\(', line)
            if abs_effect_match:
                abs_effect_avg = float(abs_effect_match.group(1))
        # In the absolute section, extract the 95% CI
        if current_section == 'absolute' and '95% CI' in line:
            ci_match = re.search(r'\[(-?\d+\.\d+),\s*(-?\d+\.\d+)\]', line)
            if ci_match:
                abs_ci_lower, abs_ci_upper = map(float, ci_match.groups())
        # Also look for the plain posterior probability if needed (not used here)
    return {
        "abs_effect_avg": abs_effect_avg,
        "abs_ci_lower": abs_ci_lower,
        "abs_ci_upper": abs_ci_upper
    }

# --------------------------
# Function to generate synthetic data with regressors and NO treatment effect
def generate_synthetic_data(seed, n_days=365, n_regressors=5, treatment_start=pd.Timestamp("2023-09-01")):
    """
    Generate a synthetic dataset:
      - A daily date series for 2023.
      - n_regressors predictors following smooth random walk-like behavior.
      - Outcome 'y' as a linear combination of the regressors plus noise.
    No treatment effect is added.
    """
    np.random.seed(seed)
    # Create date series for 2023
    dates = pd.date_range(start="2023-01-01", periods=n_days, freq="D")
    
    # Generate synthetic regressors
    regressors = {}
    for i in range(1, n_regressors + 1):
        base = np.cumsum(np.random.normal(0, 0.5, n_days))
        regressors[f"regressor_{i}"] = base + np.random.normal(0, 1, n_days)
    
    # Create DataFrame with dates and regressors
    df = pd.DataFrame({"date": dates})
    for key, val in regressors.items():
        df[key] = val

    # Generate outcome 'y'
    coefficients = np.random.uniform(0.5, 1.5, n_regressors)
    baseline_y = np.zeros(n_days)
    for i in range(n_regressors):
        baseline_y += coefficients[i] * df[f"regressor_{i+1}"]
    noise = np.random.normal(0, 2, n_days)
    baseline_y += noise

    # IMPORTANT: No treatment effect is added
    df["y"] = baseline_y

    return df

# --------------------------
# Function to run CausalImpact analysis on the simulated dataset
def run_pyci_analysis(df, treatment_start):
    """
    Run pycausalimpact on the given DataFrame.
    The DataFrame must contain a 'date' column, outcome 'y', and predictor columns.
    """
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    # Use outcome and all regressors as predictors
    predictor_cols = [col for col in df.columns if col not in ['date', 'y']]
    data = df.set_index('date')[["y"] + predictor_cols]
    
    # Define pre- and post-treatment periods.
    pre_period = [data.index.min(), treatment_start - pd.Timedelta(days=1)]
    post_period = [treatment_start, data.index.max()]
    
    # Run CausalImpact analysis
    impact = CausalImpact(data, pre_period, post_period)
    summary_text = impact.summary(output='summary')
    return parse_summary(summary_text)


In [13]:
from tqdm import tqdm 

# --------------------------
# Simulation loop to evaluate false positive rate (Type I error)
n_simulations = 500  # number of simulation runs
treatment_start = pd.Timestamp("2023-09-01")
false_positive_count = 0
results = []

for sim in tqdm(range(n_simulations)):
    # Generate synthetic data with no treatment effect using a different seed each time
    df_sim = generate_synthetic_data(seed=sim, n_days=365, n_regressors=5, treatment_start=treatment_start)
    
    # Run the pycausalimpact analysis
    parsed = run_pyci_analysis(df_sim, treatment_start)
    
    # Determine if the 95% CI for the absolute effect excludes zero.
    ci_lower = parsed["abs_ci_lower"]
    ci_upper = parsed["abs_ci_upper"]
    effect_detected = False
    if ci_lower is not None and ci_upper is not None:
        # Effect is "detected" if the entire interval lies either above or below zero.
        if ci_lower > 0 or ci_upper < 0:
            effect_detected = True

    if effect_detected:
        false_positive_count += 1
        
    results.append({
        "simulation": sim,
        "abs_effect_avg": parsed["abs_effect_avg"],
        "ci_lower": ci_lower,
        "ci_upper": ci_upper,
        "detected": effect_detected
    })

# Calculate the percentage of simulations where a nonzero effect was detected
false_positive_rate = (false_positive_count / n_simulations) * 100
print(f"Out of {n_simulations} simulations with no treatment effect, a nonzero effect was detected in {false_positive_count} cases ({false_positive_rate:.1f}%).")

# Optionally, display the results for the first few simulations.



100%|██████████| 500/500 [03:19<00:00,  2.50it/s]

Out of 500 simulations with no treatment effect, a nonzero effect was detected in 50 cases (10.0%).
